In [ ]:
import json
import pandas as pd
import numpy as np
import re

In [ ]:
# from utils import TextCleaner, CleanTwitter
import string
import warnings

from scipy import sparse
from sklearn import metrics
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.validation import check_is_fitted, check_X_y

import sys
sys.path.insert(0, "../")

from hinglishutils import print_confusion_matrix

warnings.filterwarnings("ignore")

In [ ]:
from pathlib import Path
datapath = Path("../data")
data_raw = datapath/"raw"
data_interim = datapath/"interim"
data_processed = datapath/"processed"
cleanlab_datapath = datapath/"cleanlab"

In [ ]:
# from sklearn.model_selection import train_test_split
# df = pd.read_json("data/processed/train-large.json")
# train, test = train_test_split(df, test_size=0.2)

# train.to_json("data/processed/train.json")
# test.to_json("data/processed/test.json")

In [ ]:
train = pd.read_json("data/processed/train.json")
train.head()

In [ ]:
num_classes = len(set(list(train["sentiment"])))
num_classes

In [ ]:
def cleaner(train):
    train["clean_text"] = train["text"]
    train["clean_text"] = (
        (train["clean_text"])
        .apply(lambda text: re.sub(r"RT\s@\s\w+", "", text))
        .apply(lambda text: re.sub(r"@", "mention", text))
        .apply(lambda text: re.sub(r"#", "hashtag", text))
        .apply(lambda text: re.sub(r"http\S+", "", text))
    )

    le = LabelEncoder()
    train["target"] = le.fit_transform(train["sentiment"])
    print(f"Classes : {le.classes_}")
    X_train, X_test, y_train, y_test = train_test_split(
        train["clean_text"].values,
        train["target"].values,
        test_size=0.10,
        random_state=0,
    )

    vect = TfidfVectorizer(
        strip_accents="unicode",
        ngram_range=(1, 2),
        max_df=0.75,
        min_df=3,
        sublinear_tf=True,
    )

    tfidf_train = vect.fit_transform(X_train)
    tfidf_test = vect.transform(X_test)

    return X_train, X_test, y_train, y_test, tfidf_train, tfidf_test, le.classes_

In [ ]:
%time X_train, X_test, y_train, y_test, tfidf_train, tfidf_test, label_classes = cleaner(train)

# Naive Bayes

In [ ]:
class NbSvmClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, C=1.0, dual=False, n_jobs=1):
        self.C = C
        self.dual = dual
        self.n_jobs = n_jobs

    def predict(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ["_r", "_clf"])
        return self._clf.predict(x.multiply(self._r))

    def predict_proba(self, x):
        # Verify that model has been fit
        check_is_fitted(self, ["_r", "_clf"])
        return self._clf.predict_proba(x.multiply(self._r))

    def fit(self, x, y):
        # Check that X and y have correct shape
        x, y = check_X_y(x, y, accept_sparse=True)

        def pr(x, y_i, y):
            p = x[y == y_i].sum(0)
            return (p + 1) / ((y == y_i).sum() + 1)

        self._r = sparse.csr_matrix(np.log(pr(x, 1, y) / pr(x, 0, y)))
        x_nb = x.multiply(self._r)
        self._clf = LogisticRegression(
            C=self.C, dual=self.dual, n_jobs=self.n_jobs
        ).fit(x_nb, y)
        return self

In [ ]:
NBSVM = NbSvmClassifier(C=4, dual=False, n_jobs=-1)

In [ ]:
def evaluate(model, tfidf_train, tfidf_test, y_train, y_test, s=None, with_noisy_labels= False):

    scores = cross_val_score(
        model, tfidf_train, y_train, cv=5, scoring="f1_macro"
    )
    print(f"Cross Validation Scores {scores}")
    
    if with_noisy_labels : 
        _ = model.fit(tfidf_train, s)
    else:
        _ = model.fit(tfidf_train, y_train)
    pred = model.predict(tfidf_test)
    print("Valid accuracy:", round(accuracy_score(pred, y_test), 5))
    print("Valid f1_score:", round(f1_score(pred, y_test, average="macro"), 5))
    cm = confusion_matrix(pred, y_test)
    print_confusion_matrix(cm, class_names=label_classes)

In [ ]:
evaluate(NBSVM, tfidf_train, tfidf_test, y_train, y_test)

# Logistic Regression with TF-IDF

In [ ]:
LR = LogisticRegression(C=4, dual=False, solver="lbfgs", max_iter=1000)

In [ ]:
evaluate(LR, tfidf_train, tfidf_test, y_train, y_test)

# With Cleanlab?

In [ ]:
from cleanlab.classification import LearningWithNoisyLabels
from cleanlab.noise_generation import generate_noise_matrix_from_trace
from cleanlab.noise_generation import generate_noisy_labels
from cleanlab.util import value_counts
from cleanlab.latent_algebra import compute_inv_noise_matrix
import cleanlab

In [ ]:
# Set the sparsity of the noise matrix.
FRAC_ZERO_NOISE_RATES = 0.5
# A proxy for the fraction of labels that are correct.
avg_trace = 0.95  # ~5% wrong labels. Increasing makes the problem easier.

p_y = value_counts(y_train)  # probability of y estimates
noise_matrix = generate_noise_matrix_from_trace(
    K=num_classes,
    trace=num_classes * avg_trace,
    py=p_y,
    frac_zero_noise_rates=FRAC_ZERO_NOISE_RATES,
)

# Create noisy labels
s = generate_noisy_labels(y_train, noise_matrix)

## NB-SVM Regular Train Labels 

In [ ]:
print("NB-SVM Regular Train Labels confident learning (noise matrix given),", end=" ")
NBSVM_Confident = NbSvmClassifier(C=4, dual=False, n_jobs=-1)
rp = LearningWithNoisyLabels(clf=NBSVM_Confident)
evaluate(rp, tfidf_train, tfidf_test, y_train, y_test)

## LR Regular Train Labels 

In [ ]:
print("LR Regular Train Labels confident learning (noise matrix given),", end=" ")
LR = LogisticRegression(C=4, dual=False, solver="lbfgs", max_iter=1000)
rp = LearningWithNoisyLabels(clf=LR)
evaluate(rp, tfidf_train, tfidf_test, y_train, y_test)

## NB-SVM Noisy Train Labels 

In [ ]:
print("NB-SVM Noisy Train Labels confident learning (noise matrix given),", end=" ")
NBSVM_Confident = NbSvmClassifier(C=4, dual=False, n_jobs=-1)
rp = LearningWithNoisyLabels(clf=NBSVM_Confident)
evaluate(rp, tfidf_train, tfidf_test, y_train, y_test, s = s, with_noisy_labels=True)

## LR Noisy Train Labels 

In [ ]:
print("LR Noisy Train Labels confident learning (noise matrix given),", end=" ")
LR = LogisticRegression(C=4, dual=False, solver="lbfgs", max_iter=1000)
rp = LearningWithNoisyLabels(clf=LR)
evaluate(rp, tfidf_train, tfidf_test, y_train, y_test, s=s, with_noisy_labels=True)